In [1]:
#pip install xlrd
import pandas as pd
import unidecode
import os
import glob
import numpy as np

* data is from the National Election Office of Hungary (https://www.valasztas.hu/1990-2019_eredmenyek)
* 2002,2006 and 2010 data is stored in txt files, where 1) results and 2) nominating party and 3) village id are stored in a different file
* 2014 and 2019 data is stored in a csv

In [2]:
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list

# Local elections

## 2002

In [3]:
#reading results
results = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2002_onkormanyzati/polgoj.txt',
                 delimiter="|", header=None)
#results

In [4]:
#reading nominating organization data
nominating_o = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2002_onkormanyzati/polgsz.txt',
                 delimiter="|", header=None)
#nominating_o

In [5]:
#more parties can nominate a candidate, I only keep one 
df3 = nominating_o.drop_duplicates(subset=[0,1,2], keep='first')
#df3 = df2.drop_duplicates(subset=[0,1,2], take_last=False)

In [6]:
#reading village ids
village = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2002_onkormanyzati/polgof.txt',
                 delimiter="|", header=None)
#village

In [7]:
village_key = village[[0,2,3]]
#village_key

In [8]:
elections_m = results.merge(df3, how='left', left_on=[0,1,2], right_on = [0,1,2])
#elections_m.head(10)

In [9]:
#droping redundant columns
elections_m.drop(elections_m.columns[[3, 10, 7, 11]], axis = 1, inplace = True)
#elections_m

In [10]:
elections_m = elections_m.rename(columns = {0: 'county_code', 1: 'village_code', 2: 'candidate_id',
                                           '4_x': 'candidate_name', '5_x': 'votes', '6_x': 'mandate',
                                           '4_y': 'party_id', '5_y': 'party_name'},
                                            inplace = False)
#elections_m

In [11]:
total_votes= pd.DataFrame(elections_m.groupby(['county_code', 'village_code']).sum().reset_index())
#total_votes

In [12]:
total_votes = total_votes[['county_code','village_code','votes']]
#total_votes

In [13]:
#total_votes.to_csv('total_votes.csv', index = False)

In [14]:
#total_votes = pd.read_csv('total_votes.csv')

In [15]:
elections_m2 = elections_m.merge(total_votes, how='left', left_on=['county_code','village_code'], right_on = ['county_code','village_code'])
#elections_m2.head(10)

In [16]:
#calculating the percentage of votes each candidate got
elections_m2['votes_percent'] =elections_m2['votes_x']/elections_m2['votes_y']*100
#elections_m2

In [17]:
margin = elections_m2[['county_code','village_code','votes_percent']]
#margin

In [18]:
margin_2nd =pd.DataFrame(margin.groupby(['county_code', 'village_code'])['votes_percent'].nlargest(2).reset_index())
margin_2nd['second_share'] = margin_2nd['votes_percent'] 
margin_2nd['second_share_copy'] = margin_2nd['votes_percent'] 
margin_2nd['second_share'] = margin_2nd['second_share'].shift(-1)
margin_2nd
margin_2nd = margin_2nd.loc[margin_2nd['votes_percent'] != 100]
margin_2nd = margin_2nd.loc[margin_2nd['votes_percent'] != 0]
margin_2nd['mask'] = margin_2nd.duplicated(margin_2nd.columns[[0,1]])
margin_2nd = margin_2nd.loc[margin_2nd['mask'] != True]
#margin_2nd = margin_2nd[margin_2nd.index % 2 == 0]
margin_2nd["margin"] = margin_2nd['votes_percent'] -margin_2nd['second_share']
margin_2nd['flag']=1
margin_2nd = margin_2nd[['county_code','village_code','votes_percent', 'second_share','margin','flag']]
#margin_2nd
#margin_2nd.to_csv('margin_2nd2.csv', index = False)

In [19]:
#merge margin of victory
elections_final = elections_m2.merge(margin_2nd, how='left', left_on=['county_code','village_code', 'mandate'], right_on = ['county_code','village_code', 'flag'])
#merge village names
elections_final = elections_final.merge(village_key, how='left', left_on=['county_code','village_code'], right_on = [0,2])
#only keep winners
elections_final = elections_final.loc[elections_final["mandate"] == 1]
#removing redundant columns
elections_final = elections_final.drop(columns=['votes_y', 'votes_percent_y', 'flag'])
elections_final = elections_final.drop(columns=[0,2])
#cleaning village names
elections_final.loc[elections_final[3] == 'Kömlő', 'village_name'] = "komlo2"
elections_final.loc[elections_final[3] == 'Kömörő', 'village_name'] = "komoro2"
elections_final[3] = elections_final[3].str.lower()
elections_final[3] = elections_final[3].apply(unidecode.unidecode)
elections_final.loc[elections_final['votes_percent_x'] == 100, 'margin'] = elections_final['votes_percent_x']
elections_final = elections_final.rename(columns = {3: 'village_names'},
                                            inplace = False)
elections_final['year'] = 2002
elections_final['election_type'] = 'local_elections'
elections_final.head(50)

,county_code,village_code,candidate_id,candidate_name,votes_x,mandate,party_id,party_name,votes_percent_x,second_share,margin,village_names,village_name,year,election_type
2,1,1,3,NAGY GÁBOR TAMÁS,7247,1,1039,FIDESZ-MAGYAR POLGÁRI PÁRT,46.830372,34.862682,11.967690,budapest i.ker.,NaN,2002,local_elections
6,1,2,3,HORVÁTH CSABA,22766,1,1002,MAGYAR SZOCIALISTA PÁRT,47.059553,46.912789,0.146764,budapest ii.ker.,NaN,2002,local_elections
9,1,3,3,TARLÓS ISTVÁN,30509,1,0,Független,52.779171,43.755730,9.023441,budapest iii.ker.,NaN,2002,local_elections
11,1,4,2,DERCE TAMÁS,23151,1,1058,ÚJPESTÉRT EGYESÜLET,58.553796,41.446204,17.107593,budapest iv.ker.,NaN,2002,local_elections
13,1,5,2,STEINER PÁL,7580,1,1002,MAGYAR SZOCIALISTA PÁRT,48.366514,47.626340,0.740174,budapest v.ker.,NaN,2002,local_elections
16,1,6,2,VERÓK ISTVÁN,9659,1,1002,MAGYAR SZOCIALISTA PÁRT,52.992813,47.007187,5.985626,budapest vi.ker.,NaN,2002,local_elections
19,1,7,3,HUNVALD GYÖRGY,14444,1,1002,MAGYAR SZOCIALISTA PÁRT,59.866540,30.646164,29.220376,budapest vii.ker.,NaN,2002,local_elections
22,1,8,1,CSÉCSEI BÉLA,15128,1,1002,MAGYAR SZOCIALISTA PÁRT,57.921740,36.346581,21.575159,budapest viii.ker.,NaN,2002,local_elections
26,1,9,2,GEGESY FERENC,16577,1,1035,SZABAD DEMOKRATÁK SZÖVETSÉGE,68.460395,31.539605,36.920790,budapest ix.ker.,NaN,2002,local_elections
27,1,10,1,ANDÓ SÁNDOR,15901,1,1002,MAGYAR SZOCIALISTA PÁRT,53.734117,42.680454,11.053663,budapest x.ker.,NaN,2002,local_elections


In [20]:
elections_final['party_name'] = elections_final['party_name'].str.lower()
elections_final['party_name'] = elections_final['party_name'].apply(unidecode.unidecode)
elections_final['party_name'] = elections_final['party_name'].str.strip()
# create a choice list
choices = elections_final['party_name'].values.tolist()
Remove(choices)

['fidesz-magyar polgari part',
 'magyar szocialista part',
 'fuggetlen',
 'ujpestert egyesulet',
 'szabad demokratak szovetsege',
 'soroksari civil szervezetek',
 'bolyi kulturalis egyesulet',
 'egysegben harkanyert',
 'zsongorko barati kor',
 'munkaspart',
 'keresztenydemokrata neppart',
 'magyar demokrata forum',
 'egyutt kiskunmajsaert',
 'melykuti gazdakor',
 '"vago janos" gazdakor',
 'hagyomanyorzo es faluszepito e',
 'kotegyani barati kor egyesulet',
 'osszefogas bekes megyeert',
 'nyugdijas egyesulet',
 'zempleni telepulesszovetseg',
 'vallalkozok barati egyesulet',
 'centrum',
 'magyar igazsag es elet partja',
 '"putnok varosert" egyesulet',
 'polgarmesterek fejer megyeert',
 'racalmasi faluvedo egyesulet',
 'rackereszturi nepkor egyesulet',
 'videkert egyesulet',
 'htfe',
 'klub 2000 barati egyesulet',
 'lorinci varos jovojeert b.e.',
 'firosz',
 'turkeven elokert egyesulet',
 'faluvedo egyesulet tardos',
 'agrarszovetseg',
 'mkdsz',
 'budakesziert fuggetlenek egy.',
 'demokra

In [21]:
#final cleaning

elections_final2002=elections_final.copy()
elections_final2002= elections_final2002[['candidate_name', 'votes_x', 'party_name', 'votes_percent_x', 'second_share', 'margin', 'village_names', 'year', 'election_type']]
elections_final2002 = elections_final2002.rename(columns = {'votes_x': 'votes_winner', 'votes_percent_x': 'winner_share', 'village_names':'village_name'},
                                            inplace = False)
elections_final2002['village_name'] = elections_final2002['village_name'].str.replace("ker.", "", case = False)
fidesz = pd.Series(['fidesz-magyar polgari part','magyar demokrata forum', 'keresztenydemokrata neppart'])
mszp = pd.Series(['magyar szocialista part','szabad demokratak szovetsege'])
independent = pd.Series(['fuggetlen'])
elections_final2002['fidesz_list'] = pd.Categorical(elections_final2002['party_name'].isin(fidesz).astype(np.int8))
elections_final2002['mszp_list'] = pd.Categorical(elections_final2002['party_name'].isin(mszp).astype(np.int8))
elections_final2002['governing_pary'] = pd.Categorical(elections_final2002['party_name'].isin(mszp).astype(np.int8))
elections_final2002['independent'] = pd.Categorical(elections_final2002['party_name'].isin(independent).astype(np.int8))
elections_final2002

,candidate_name,votes_winner,party_name,winner_share,second_share,margin,village_name,year,election_type,fidesz_list,mszp_list,governing_pary,independent
2,NAGY GÁBOR TAMÁS,7247,fidesz-magyar polgari part,46.830372,34.862682,11.967690,budapest i.,2002,local_elections,1,0,0,0
6,HORVÁTH CSABA,22766,magyar szocialista part,47.059553,46.912789,0.146764,budapest ii.,2002,local_elections,0,1,1,0
9,TARLÓS ISTVÁN,30509,fuggetlen,52.779171,43.755730,9.023441,budapest iii.,2002,local_elections,0,0,0,1
11,DERCE TAMÁS,23151,ujpestert egyesulet,58.553796,41.446204,17.107593,budapest iv.,2002,local_elections,0,0,0,0
13,STEINER PÁL,7580,magyar szocialista part,48.366514,47.626340,0.740174,budapest v.,2002,local_elections,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8262,ÁBRAHÁM ALADÁR,216,fuggetlen,53.731343,24.626866,29.104478,zalatarnok,2002,local_elections,0,0,0,1
8267,SZABÓ JÓZSEF,60,fuggetlen,81.081081,18.918919,62.162162,zalaujlak,2002,local_elections,0,0,0,1
8268,BERKENYÉS ISTVÁN,297,fuggetlen,57.115385,18.076923,39.038462,zalavar,2002,local_elections,0,0,0,1
8273,NAGY TAMÁS,178,fuggetlen,66.666667,33.333333,33.333333,zalaveg,2002,local_elections,0,0,0,1


In [22]:
elections_final2002.to_csv('elections_local_2002.csv', index = False)

## 2006

In [23]:
#reading data
#results
results = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2006_onkormanyzati/polgoj.txt',
                 delimiter="|", header=None)

#nominating organization data
nominating_o = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2006_onkormanyzati/polgsz.txt',
                 delimiter="|", header=None)

#more parties can nominate a candidate, I only keep one 
df3 = nominating_o.drop_duplicates(subset=[0,1,2], keep='first')
#df3 = df2.drop_duplicates(subset=[0,1,2], take_last=False)

#village ids
village = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2006_onkormanyzati/polgof.txt',
                 delimiter="|", header=None)
village_key = village[[0,2,3]]

In [24]:
#merging results and nominating organization data
elections_m = results.merge(df3, how='left', left_on=[0,1,2], right_on = [0,1,2])
elections_m

,0,1,2,3_x,4_x,5_x,6,7,8,3_y,4_y,5_y
0,1,1,1,Dr.,Nagy Gábor Tamás ...,8967,1,0,,1,86,FIDESZ - MAGYAR POLGÁRI SZÖVETSÉG ...
1,1,1,2,,Rotyis József ...,5057,0,0,,1,7,MAGYAR SZOCIALISTA PÁRT ...
2,1,1,3,Dr.,Vaskor László János ...,993,0,0,,1,14,MAGYAR DEMOKRATA FÓRUM ...
3,1,2,1,,Horváth Csaba ...,21216,0,0,,1,7,MAGYAR SZOCIALISTA PÁRT ...
4,1,2,2,Dr.,Láng Zsolt ...,25445,1,0,,1,86,FIDESZ - MAGYAR POLGÁRI SZÖVETSÉG ...
...,...,...,...,...,...,...,...,...,...,...,...,...
8578,20,255,2,,Gábor Csaba Sándor ...,50,0,0,,0,0,Független ...
8579,20,255,3,,Lucz József ...,67,0,0,,0,0,Független ...
8580,20,256,1,,Nagy Tamás ...,95,0,0,,0,0,Független ...
8581,20,256,2,,Pék József Csaba ...,165,1,0,,0,0,Független ...


In [25]:
#droping redundant columns
elections_m.drop(elections_m.columns[[3, 7, 8, 9]], axis = 1, inplace = True)
elections_m

,0,1,2,4_x,5_x,6,4_y,5_y
0,1,1,1,Nagy Gábor Tamás ...,8967,1,86,FIDESZ - MAGYAR POLGÁRI SZÖVETSÉG ...
1,1,1,2,Rotyis József ...,5057,0,7,MAGYAR SZOCIALISTA PÁRT ...
2,1,1,3,Vaskor László János ...,993,0,14,MAGYAR DEMOKRATA FÓRUM ...
3,1,2,1,Horváth Csaba ...,21216,0,7,MAGYAR SZOCIALISTA PÁRT ...
4,1,2,2,Láng Zsolt ...,25445,1,86,FIDESZ - MAGYAR POLGÁRI SZÖVETSÉG ...
...,...,...,...,...,...,...,...,...
8578,20,255,2,Gábor Csaba Sándor ...,50,0,0,Független ...
8579,20,255,3,Lucz József ...,67,0,0,Független ...
8580,20,256,1,Nagy Tamás ...,95,0,0,Független ...
8581,20,256,2,Pék József Csaba ...,165,1,0,Független ...


In [26]:
elections_m = elections_m.rename(columns = {0: 'county_code', 1: 'village_code', 2: 'candidate_id',
                                           '4_x': 'candidate_name', '5_x': 'votes', 6: 'mandate',
                                           '4_y': 'party_id', '5_y': 'party_name',},
                                            inplace = False)
#elections_m

In [27]:
#calculating margine of victory (percentage point difference between the first and second)
#calculate total number of votes per village
total_votes= pd.DataFrame(elections_m.groupby(['county_code', 'village_code']).sum().reset_index())
#removing redundant columns
total_votes = total_votes[['county_code','village_code','votes']]

#merging village-level data to the candidate-level data
elections_m2 = elections_m.merge(total_votes, how='left', left_on=['county_code','village_code'], right_on = ['county_code','village_code'])

#calculating the percentage of votes each candidate got
elections_m2['votes_percent'] =elections_m2['votes_x']/elections_m2['votes_y']*100

#keeping relevant columns only
margin = elections_m2[['county_code','village_code','votes_percent']]

#calculating margin of victory
margin_2nd =pd.DataFrame(margin.groupby(['county_code', 'village_code'])['votes_percent'].nlargest(2).reset_index())
margin_2nd['second_share'] = margin_2nd['votes_percent'] 
margin_2nd['second_share_copy'] = margin_2nd['votes_percent'] 
margin_2nd['second_share'] = margin_2nd['second_share'].shift(-1)
margin_2nd = margin_2nd.loc[margin_2nd['votes_percent'] != 100]
margin_2nd = margin_2nd.loc[margin_2nd['votes_percent'] != 0]
margin_2nd['mask'] = margin_2nd.duplicated(margin_2nd.columns[[0,1]])
margin_2nd = margin_2nd.loc[margin_2nd['mask'] != True]
margin_2nd["margin"] = margin_2nd['votes_percent'] -margin_2nd['second_share']
margin_2nd['flag']=1
margin_2nd = margin_2nd[['county_code','village_code','votes_percent', 'second_share','margin','flag']]
#margin_2nd

In [28]:
#merge margin of victory
elections_final = elections_m2.merge(margin_2nd, how='left', left_on=['county_code','village_code', 'mandate'], right_on = ['county_code','village_code', 'flag'])
#merge village names
elections_final = elections_final.merge(village_key, how='left', left_on=['county_code','village_code'], right_on = [0,2])
#only keep winners
elections_final = elections_final.loc[elections_final["mandate"] == 1]
#removing redundant columns
elections_final = elections_final.drop(columns=['votes_y', 'votes_percent_y', 'flag'])
elections_final = elections_final.drop(columns=[0,2])
#cleaning village names
elections_final.loc[elections_final[3] == 'Kömlő', 'village_name'] = "komlo2"
elections_final.loc[elections_final[3] == 'Kömörő', 'village_name'] = "komoro2"
elections_final[3] = elections_final[3].str.lower()
elections_final[3] = elections_final[3].apply(unidecode.unidecode)
elections_final.loc[elections_final['votes_percent_x'] == 100, 'margin'] = elections_final['votes_percent_x']
elections_final = elections_final.rename(columns = {3: 'village_names'},
                                            inplace = False)
elections_final['party_name'] = elections_final['party_name'].str.lower()
elections_final['party_name'] = elections_final['party_name'].apply(unidecode.unidecode)
elections_final['party_name'] = elections_final['party_name'].str.strip()
elections_final['year'] = 2006
elections_final['election_type'] = 'local_elections'
#elections_final

In [29]:
# create a choice list
choices = elections_final['party_name'].values.tolist()
#choices
Remove(choices)

['fidesz - magyar polgari szovetseg',
 'ujpestert egyesulet',
 'magyar szocialista part',
 'szdsz- a magyar liberalis part',
 'soroksari civil szervezetek es szemelyek egyesulete',
 'fuggetlen',
 'magyar demokrata forum',
 'az europai unio roma ifjusagaert egyesulet',
 'egysegben harkanyert',
 'kisgazda polgari egyesulet',
 'zsongorko barati kor',
 'keresztenydemokrata neppart',
 'vago janos gazdakor',
 'hagyomanyorzo es faluszepito egyesulet',
 'sarkadi nyugdijasok erdekvedelmi egyesulete',
 'zempleni telepulesszovetseg',
 'magyar kommunista munkaspart',
 'nemzeti forum egyesulet',
 'roma polgarjogi mozgalom b.-a.-z. megyei szervezete',
 'magyar igazsag es elet partja',
 '"putnok varosert" egyesulet',
 'polgarmesterek fejer megyeert',
 'fejer megyei telepulesekert egyesulet',
 'rackereszturi nepkor egyesulet',
 'biharnagybajomert egyesulet',
 'hajduboszormenyi polgari szovetseg egyesulet',
 'osszefogas hajdudorogert egyesulet',
 'hajdunanasert tevekenykedok fuggetlen egyesulete',
 'os

In [30]:
#final cleaning
elections_final2006=elections_final.copy()
elections_final2006= elections_final2006[['candidate_name', 'votes_x', 'party_name', 'votes_percent_x', 'second_share', 'margin', 'village_names', 'year', 'election_type']]
elections_final2006 = elections_final2006.rename(columns = {'votes_x': 'votes_winner', 'votes_percent_x': 'winner_share', "village_names":'village_name'},
                                            inplace = False)
elections_final2006['village_name'] = elections_final2006['village_name'].str.replace("ker.", "", case = False)
fidesz = pd.Series(['fidesz - magyar polgari szovetseg', 'keresztenydemokrata neppart'])
mszp = pd.Series(['magyar szocialista part','szdsz- a magyar liberalis part'])
independent = pd.Series(['fuggetlen'])
elections_final2006['fidesz_list'] = pd.Categorical(elections_final2006['party_name'].isin(fidesz).astype(np.int8))
elections_final2006['mszp_list'] = pd.Categorical(elections_final2006['party_name'].isin(mszp).astype(np.int8))
elections_final2006['governing_pary'] = pd.Categorical(elections_final2006['party_name'].isin(mszp).astype(np.int8))
elections_final2006['independent'] = pd.Categorical(elections_final2006['party_name'].isin(independent).astype(np.int8))
elections_final2006.head(50)

,candidate_name,votes_winner,party_name,winner_share,second_share,margin,village_name,year,election_type,fidesz_list,mszp_list,governing_pary,independent
0,Nagy Gábor Tamás ...,8967,fidesz - magyar polgari szovetseg,59.712326,33.675168,26.037158,budapest i.,2006,local_elections,1,0,0,0
4,Láng Zsolt ...,25445,fidesz - magyar polgari szovetseg,52.153149,43.485212,8.667937,budapest ii.,2006,local_elections,1,0,0,0
6,Bús Balázs ...,27282,fidesz - magyar polgari szovetseg,44.229366,39.406319,4.823047,budapest iii.,2006,local_elections,1,0,0,0
9,Derce Tamás ...,21272,ujpestert egyesulet,51.283782,30.478073,20.805709,budapest iv.,2006,local_elections,0,0,0,0
13,Rogán Antal ...,7857,fidesz - magyar polgari szovetseg,51.738443,44.982220,6.756223,budapest v.,2006,local_elections,1,0,0,0
18,Verók István ...,7920,magyar szocialista part,45.010230,43.242782,1.767447,budapest vi.,2006,local_elections,0,1,1,0
19,Hunvald György ...,12815,magyar szocialista part,52.867162,40.585809,12.281353,budapest vii.,2006,local_elections,0,1,1,0
23,Csécsei Béla ...,13391,szdsz- a magyar liberalis part,50.480642,41.482263,8.998379,budapest viii.,2006,local_elections,0,1,1,0
27,Gegesy Ferenc ...,9495,szdsz- a magyar liberalis part,40.175171,34.898875,5.276297,budapest ix.,2006,local_elections,0,1,1,0
34,Verbai Lajos ...,14970,magyar szocialista part,50.110464,48.483631,1.626833,budapest x.,2006,local_elections,0,1,1,0


In [31]:
#saving data
elections_final2006.to_csv('elections_local_2006.csv', index = False)

## 2010

In [32]:
#reading data
#results
results = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2010_onkormanyzati/polgoj.txt',
                 delimiter="|", header=None)

#nominating organization data
nominating_o = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2010_onkormanyzati/polgsz.txt',
                 delimiter="|", header=None)

#more parties can nominate a candidate, I only keep one 
df3 = nominating_o.drop_duplicates(subset=[0,1,2], keep='first')
#df3 = df2.drop_duplicates(subset=[0,1,2], take_last=False)

#village ids
village = pd.read_fwf('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2010_onkormanyzati/polgof.txt',
                 delimiter="|", header=None)
village_key = village[[0,2,3]]

In [33]:
#merging results and nominating organization data
elections_m = results.merge(df3, how='left', left_on=[0,1,2], right_on = [0,1,2])
#elections_m.head(10) 

In [34]:
#droping redundant columns
elections_m.drop(elections_m.columns[[3, 7, 8, 9]], axis = 1, inplace = True)
#elections_m

In [35]:
elections_m = elections_m.rename(columns = {0: 'county_code', 1: 'village_code', 2: 'candidate_id',
                                           '4_x': 'candidate_name', '5_x': 'votes', 6: 'mandate',
                                           '4_y': 'party_id', '5_y': 'party_name',},
                                            inplace = False)
#elections_m 

In [36]:
#calculating margine of victory (percentage point difference between the first and second)
#calculate total number of votes per village
total_votes= pd.DataFrame(elections_m.groupby(['county_code', 'village_code']).sum().reset_index())
#removing redundant columns
total_votes = total_votes[['county_code','village_code','votes']]

#merging village-level data to the candidate-level data
elections_m2 = elections_m.merge(total_votes, how='left', left_on=['county_code','village_code'], right_on = ['county_code','village_code'])

#calculating the percentage of votes each candidate got
elections_m2['votes_percent'] =elections_m2['votes_x']/elections_m2['votes_y']*100

#keeping relevant columns only
margin = elections_m2[['county_code','village_code','votes_percent']]

#calculating margin of victory
margin_2nd =pd.DataFrame(margin.groupby(['county_code', 'village_code'])['votes_percent'].nlargest(2).reset_index())
margin_2nd['second_share'] = margin_2nd['votes_percent'] 
margin_2nd['second_share_copy'] = margin_2nd['votes_percent'] 
margin_2nd['second_share'] = margin_2nd['second_share'].shift(-1)
margin_2nd = margin_2nd.loc[margin_2nd['votes_percent'] != 100]
margin_2nd = margin_2nd.loc[margin_2nd['votes_percent'] != 0]
margin_2nd['mask'] = margin_2nd.duplicated(margin_2nd.columns[[0,1]])
margin_2nd = margin_2nd.loc[margin_2nd['mask'] != True]
margin_2nd["margin"] = margin_2nd['votes_percent'] -margin_2nd['second_share']
margin_2nd['flag']=1
margin_2nd = margin_2nd[['county_code','village_code','votes_percent', 'second_share','margin','flag']]
#margin_2nd

In [37]:
#merge margin of victory
elections_final = elections_m2.merge(margin_2nd, how='left', left_on=['county_code','village_code', 'mandate'], right_on = ['county_code','village_code', 'flag'])

#merge village names
elections_final = elections_final.merge(village_key, how='left', left_on=['county_code','village_code'], right_on = [0,2])

#only keep winners
elections_final = elections_final.loc[elections_final["mandate"] == 1]

#removing redundant columns
elections_final = elections_final.drop(columns=['votes_y', 'votes_percent_y', 'flag'])
elections_final = elections_final.drop(columns=[0,2])

#cleaning village names
elections_final.loc[elections_final['votes_percent_x'] == 100, 'margin'] = elections_final['votes_percent_x']
elections_final = elections_final.rename(columns = {3: 'village_name'},
                                            inplace = False)
elections_final.loc[elections_final.village_name == 'Kömlő', 'village_name'] = "komlo2"
elections_final.loc[elections_final.village_name == 'Kömörő', 'village_name'] = "komoro2"
elections_final['village_name'] = elections_final['village_name'].str.lower()
elections_final['village_name'] = elections_final['village_name'].apply(unidecode.unidecode)
elections_final['year'] = 2010
elections_final['election_type'] = 'local_elections'
#elections_final.head(10)

In [38]:
elections_final['party_name'] = elections_final['party_name'].str.lower()
elections_final['party_name'] = elections_final['party_name'].apply(unidecode.unidecode)
elections_final['party_name'] = elections_final['party_name'].str.strip()
# create a choice list
choices = elections_final['party_name'].values.tolist()
Remove(choices)

['fidesz - magyar polgari szovetseg',
 'magyar szocialista part',
 'soroksari civil szervezetek, szemelyek egyesulete',
 'fuggetlen',
 'birjanert barati kor egyesulet',
 'jovonk halas egyesulet',
 'egyutt kiskunmajsaert egyesulet',
 'nemzeti forum egyesulet',
 'szentkiralyi vago janos gazdakor',
 'gadorosi hagyomanyorzo es faluszepito egyesulet',
 'vesztoert varosfejlesztesi egyesulet',
 'magyar kommunista munkaspart',
 'magyar igazsag es elet partja',
 'osszefogas zsomboert kozhasznu egyesulet',
 'polgari osszefogas perkataert egyesulet',
 'racalmasi varosvedo es szepito egyesulet',
 'rackereszturi nepkor egyesulet',
 'csornai korona polgari egyuttmukodesi egyesulet',
 'osszefogas hajduszoboszloert es a jovoert  egyesulet',
 'jobbik magyarorszagert mozgalom',
 'magyarorszagi munkaspart 2006',
 'osszefogas fuzesabonyert egyesulet',
 'lehet mas a politika',
 'lorinci varos jovojeert barati egyesulet',
 'egyutt maklarert egyesulet',
 'osszefogas abadszalokert egyesulet',
 'fiatal romak o

In [39]:
#final cleaning
elections_final2010=elections_final.copy()
elections_final2010= elections_final2010[['candidate_name', 'votes_x', 'party_name', 'votes_percent_x', 'second_share', 'margin', 'village_name', 'year', 'election_type']]
elections_final2010 = elections_final2010.rename(columns = {'votes_x': 'votes_winner', 'votes_percent_x': 'winner_share'},
                                            inplace = False)
elections_final2010['village_name'] = elections_final2010['village_name'].str.replace("ker.", "", case = False)
fidesz = pd.Series(['fidesz - magyar polgari szovetseg', 'keresztenydemokrata neppart'])
mszp = pd.Series(['magyar szocialista part','szdsz- a magyar liberalis part'])
independent = pd.Series(['fuggetlen'])
elections_final2010['fidesz_list'] = pd.Categorical(elections_final2010['party_name'].isin(fidesz).astype(np.int8))
elections_final2010['mszp_list'] = pd.Categorical(elections_final2010['party_name'].isin(mszp).astype(np.int8))
elections_final2010['governing_pary'] = pd.Categorical(elections_final2010['party_name'].isin(fidesz).astype(np.int8))
elections_final2010['independent'] = pd.Categorical(elections_final2010['party_name'].isin(independent).astype(np.int8))
elections_final2010.head(50)

,candidate_name,votes_winner,party_name,winner_share,second_share,margin,village_name,year,election_type,fidesz_list,mszp_list,governing_pary,independent
2,Nagy Gábor Tamás ...,6957,fidesz - magyar polgari szovetseg,61.376268,18.941332,42.434936,budapest i.,2010,local_elections,1,0,1,0
5,Láng Zsolt ...,24170,fidesz - magyar polgari szovetseg,66.366457,22.740877,43.625580,budapest ii.,2010,local_elections,1,0,1,0
9,Bús Balázs ...,25580,fidesz - magyar polgari szovetseg,57.382565,26.154157,31.228409,budapest iii.,2010,local_elections,1,0,1,0
13,Wintermantel Zsolt ...,14974,fidesz - magyar polgari szovetseg,47.082128,44.117092,2.965036,budapest iv.,2010,local_elections,1,0,1,0
15,Rogán Antal ...,7870,fidesz - magyar polgari szovetseg,72.971720,18.766806,54.204914,budapest v.,2010,local_elections,1,0,1,0
18,Hassay Zsófia ...,6077,fidesz - magyar polgari szovetseg,49.725882,32.935112,16.790770,budapest vi.,2010,local_elections,1,0,1,0
24,Vattamány Zsolt ...,8185,fidesz - magyar polgari szovetseg,48.480720,29.858437,18.622283,budapest vii.,2010,local_elections,1,0,1,0
26,Kocsis Máté ...,10516,fidesz - magyar polgari szovetseg,58.009709,23.654016,34.355693,budapest viii.,2010,local_elections,1,0,1,0
29,Bácskai János ...,8396,fidesz - magyar polgari szovetseg,46.023132,39.379488,6.643644,budapest ix.,2010,local_elections,1,0,1,0
33,Kovács Róbert ...,10677,fidesz - magyar polgari szovetseg,48.952363,29.773967,19.178396,budapest x.,2010,local_elections,1,0,1,0


In [40]:
#saving data
elections_final2010.to_csv('elections_local_2010.csv', index = False)

## 2014

In [41]:
#reading data
results = pd.read_csv('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2014_onkormanyzati/polgármester_választás.csv', engine='python')
results.head(10)

,SZAVAZÓKÖR_AZON,JKV_AZONOSÍTÓ,VÁLASZTÁS_TÍPUS,MEGYEKÓD,MEGYE,TELEPÜLÉS_SORSZÁM,TELEPÜLÉS,SZAVAZÓKÖR,VÁLASZTÓPOLGÁROK_SZÁMA,MEGJELENTEK_SZÁMA,URNÁBAN_LEVŐ_NEM_BÉLYEGZETT,URNÁBAN_LEVŐ_BÉLYEGZETT,ELTÉRÉS,ÉRVÉNYTELEN_SZAVAZÓLAP,ÉRVÉNYES_SZAVAZÓLAP,SZAVLAP_SORSZÁM,JELÖLT,SZERVEZET,SZAVAZAT
0,01-001-001-F,214549,Polgármester,1.0,BUDAPEST,1.0,Budapest I. kerület,1.0,1136.0,649.0,0.0,650.0,1.0,27.0,623.0,NaN,NaN,NaN,NaN
1,01-001-001-T,214549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Remenyik Ildikó,MSZP,36.0
2,01-001-001-T,214549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Dr. Gallowich Margit Kornélia,LMP,53.0
3,01-001-001-T,214549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Dr. Kakuk György András,DK,48.0
4,01-001-001-T,214549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Váradiné Naszályi Márta,EGYÜTT-PM,58.0
5,01-001-001-T,214549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Miseje Balázs,JOBBIK,73.0
6,01-001-001-T,214549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Dr. Nagy Gábor Tamás,FIDESZ-KDNP,355.0
7,01-001-002-F,214550,Polgármester,1.0,BUDAPEST,1.0,Budapest I. kerület,2.0,1083.0,571.0,0.0,570.0,-1.0,2.0,568.0,NaN,NaN,NaN,NaN
8,01-001-002-T,214550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Remenyik Ildikó,MSZP,42.0
9,01-001-002-T,214550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Dr. Gallowich Margit Kornélia,LMP,51.0


In [42]:
#cleaning data
#droping redundant columns
results.drop(labels=['SZAVAZÓKÖR_AZON', 'JKV_AZONOSÍTÓ', 'VÁLASZTÁS_TÍPUS', 'MEGYE', 'SZAVAZÓKÖR','VÁLASZTÓPOLGÁROK_SZÁMA',
                            'MEGJELENTEK_SZÁMA', 'URNÁBAN_LEVŐ_NEM_BÉLYEGZETT', 'URNÁBAN_LEVŐ_BÉLYEGZETT', 'ELTÉRÉS', 
                             'ÉRVÉNYTELEN_SZAVAZÓLAP', 'ÉRVÉNYES_SZAVAZÓLAP', 'SZAVLAP_SORSZÁM'], axis = 1, inplace = True)
#data cleaning
elections_m = results.rename(columns = {'MEGYEKÓD': 'county_code', 'TELEPÜLÉS_SORSZÁM': 'village_code',
                                            'TELEPÜLÉS': 'village_name', 'JELÖLT': 'candidate_name', 
                                            'SZERVEZET': 'party', 'SZAVAZAT': 'votes'},
                                            inplace = False)

elections_m["votes"].fillna(0, inplace=True)
elections_m["village_name"]=elections_m["village_name"].fillna(method='ffill')
elections_m["village_code"]=elections_m["village_code"].fillna(method='ffill')
elections_m["county_code"]=elections_m["county_code"].fillna(method='ffill')
elections_m = elections_m.dropna(subset=['candidate_name'])

In [43]:
#village key
key = elections_m[['county_code', "village_code", "village_name"]]
key = key.drop_duplicates(keep='first')

#party key
key2 = elections_m[["village_name",'candidate_name', 'party']]
key2 = key2.drop_duplicates(keep='first')

In [44]:
#group by village and candiate name
total_votes = elections_m.groupby(['village_name','candidate_name']).sum().reset_index()
total_votes = total_votes.drop(columns=['county_code', 'village_code'])

#total_votes.loc[total_votes['village_name']=='Budapest I. kerület']

total_votes_merged = total_votes.merge(key, how='left', on='village_name')
total_votes_merged = total_votes_merged.merge(key2, how='left', on=['village_name', 'candidate_name'])

#cleaning
total_votes_merged.loc[total_votes_merged.village_name == 'Kömlő', 'village_name'] = "komlo2"
total_votes_merged.loc[total_votes_merged.village_name == 'Kömörő', 'village_name'] = "komoro2"

total_votes_merged['village_name'] = total_votes_merged['village_name'].str.lower()
total_votes_merged['village_name'] = total_votes_merged['village_name'].apply(unidecode.unidecode)

In [45]:
#total votes per village
total_votes2= pd.DataFrame(total_votes_merged.groupby(['county_code', 'village_code']).sum().reset_index())
total_votes2_merged = total_votes_merged.merge(total_votes2, how='left', on=['county_code', 'village_code'])

#maximum
forMax = total_votes2_merged[['county_code', "village_name", "votes_x"]]
maximums = forMax.groupby(['county_code', 'village_name']).max().reset_index()
maximums['mandate'] = 1

#calculating votes share
total_votes3_merged = total_votes2_merged.merge(maximums, how='left', on=['village_name', "votes_x"])
total_votes3_merged['votes_percent'] =total_votes3_merged['votes_x']/total_votes3_merged['votes_y']*100

#calculating the percentage of votes each candidate got
margin = total_votes3_merged[['county_code_x','village_code','votes_percent', "candidate_name", "village_name", "mandate"]]
margin = margin.loc[margin['votes_percent'] != 100]
margin = margin.loc[margin['votes_percent'] != 0]
margin_2nd =pd.DataFrame(margin.groupby(['county_code_x', 'village_code'])['votes_percent'].nlargest(2).reset_index())
margin_2nd['second_share'] = margin_2nd['votes_percent'] 
margin_2nd['second_share_copy'] = margin_2nd['votes_percent'] 
margin_2nd['second_share'] = margin_2nd['second_share'].shift(-1)
margin_2nd['mask'] = margin_2nd.duplicated(margin_2nd.columns[[0,1]])
margin_2nd = margin_2nd.loc[margin_2nd['mask'] != True]
##margin_2nd = margin_2nd[margin_2nd.index % 2 == 0]
margin_2nd["margin"] = margin_2nd['votes_percent'] -margin_2nd['second_share']
margin_2nd['flag']=1
margin_2nd = margin_2nd[['county_code_x','village_code','votes_percent', 'second_share','margin','flag']]
#margin

#margin_2nd.loc[margin_2nd['margin'] <0]

#merge margin of victory
elections_final = total_votes3_merged.merge(margin_2nd, how='left', left_on=['county_code_x','village_code', 'mandate'], right_on = ['county_code_x','village_code', 'flag'])

#only keep winners
elections_final = elections_final.loc[elections_final["mandate"] == 1]

#removing redundant columns
elections_final = elections_final.drop(columns=['votes_y', 'votes_percent_y', 'flag'])

#cleaning village names
elections_final.loc[elections_final['votes_percent_x'] == 100, 'margin'] = elections_final['votes_percent_x']
elections_final = elections_final.rename(columns = {3: 'votes_winner', 'votes_percent_x': 'votes_share_winner'}, inplace = False)
elections_final['year'] = 2014
elections_final['election_type'] = 'local_elections'
elections_final.head(50)

#elections_final.loc[elections_final['village_name']=='budapest i. kerulet']

,village_name,candidate_name,votes_x,county_code_x,village_code,party,county_code_y,mandate,votes_share_winner,second_share,margin,year,election_type
1,aba,Kossa Lajos,744.0,7.0,1.0,Független jelölt,7.0,1.0,69.597755,30.402245,39.195510,2014,local_elections
3,abaliget,Kisfali János László,217.0,2.0,1.0,Független jelölt,2.0,1.0,58.648649,41.351351,17.297297,2014,local_elections
4,abasar,Dr. Lénártné Benei Anikó,714.0,10.0,1.0,Független jelölt,10.0,1.0,55.868545,44.131455,11.737089,2014,local_elections
7,abaujalpar,Zinger Zsolt,37.0,5.0,1.0,Független jelölt,5.0,1.0,100.000000,NaN,100.000000,2014,local_elections
10,abaujker,Szabó Gábor,149.0,5.0,2.0,Független jelölt,5.0,1.0,39.839572,30.213904,9.625668,2014,local_elections
12,abaujlak,Pollák István Zoltán,63.0,5.0,3.0,Független jelölt,5.0,1.0,69.230769,30.769231,38.461538,2014,local_elections
14,abaujszolnok,Gönczi Gáborné,61.0,5.0,5.0,Független jelölt,5.0,1.0,65.591398,29.032258,36.559140,2014,local_elections
18,abaujszanto,Kiss Tibor István,646.0,5.0,4.0,Független jelölt,5.0,1.0,41.463415,28.626444,12.836970,2014,local_elections
22,abaujvar,Sivák Tamás,69.0,5.0,6.0,Független jelölt,5.0,1.0,43.949045,41.401274,2.547771,2014,local_elections
23,abda,Szabó Zsolt,890.0,8.0,1.0,Független jelölt,8.0,1.0,77.933450,22.066550,55.866900,2014,local_elections


In [46]:
elections_final['party'] = elections_final['party'].str.lower()
elections_final['party'] = elections_final['party'].apply(unidecode.unidecode)
elections_final['party'] = elections_final['party'].str.strip()
# create a choice list
choices = elections_final['party'].values.tolist()
#Remove(choices)

In [47]:
#final cleaning
elections_final2014=elections_final.copy()
elections_final2014= elections_final2014[['candidate_name', 'votes_x', 'party', 'votes_share_winner', 'second_share', 'margin', 'village_name', 'year', 'election_type']]
elections_final2014 = elections_final2014.rename(columns = {'votes_x': 'votes_winner', 'votes_share_winner': 'winner_share', 'party':'party_name'},
                                            inplace = False)
elections_final2014['village_name'] = elections_final2014['village_name'].str.replace("kerulet", "", case = False)
fidesz = pd.Series(['fidesz-kdnp', 'fidesz',
                    'fidesz-faddi pe','fidesz-kdnp-kpe','fidesz-nagykun polgari k','fidesz-kdnp-martoni mas kep',
 'fidesz-kdnp-nemzeti forum-mpke-labe',
 'fidesz-kdnp-nemzeti forum',
 'pope-fidesz-kdnp',
 'nemzeti forum-fidesz',
 'fidesz-kdnp-pve',
 'fidesz-kdnp-otvsz',
 'fidesz-kdnp-gazdakor'])
independent = pd.Series(['fuggetlen jelolt'])
elections_final2014['fidesz_list'] = pd.Categorical(elections_final2014['party_name'].isin(fidesz).astype(np.int8))
elections_final2014['governing_pary'] = pd.Categorical(elections_final2014['party_name'].isin(fidesz).astype(np.int8))
elections_final2014['independent'] = pd.Categorical(elections_final2014['party_name'].isin(independent).astype(np.int8))
elections_final2014.head(50)

,candidate_name,votes_winner,party_name,winner_share,second_share,margin,village_name,year,election_type,fidesz_list,governing_pary,independent
1,Kossa Lajos,744.0,fuggetlen jelolt,69.597755,30.402245,39.195510,aba,2014,local_elections,0,0,1
3,Kisfali János László,217.0,fuggetlen jelolt,58.648649,41.351351,17.297297,abaliget,2014,local_elections,0,0,1
4,Dr. Lénártné Benei Anikó,714.0,fuggetlen jelolt,55.868545,44.131455,11.737089,abasar,2014,local_elections,0,0,1
7,Zinger Zsolt,37.0,fuggetlen jelolt,100.000000,NaN,100.000000,abaujalpar,2014,local_elections,0,0,1
10,Szabó Gábor,149.0,fuggetlen jelolt,39.839572,30.213904,9.625668,abaujker,2014,local_elections,0,0,1
12,Pollák István Zoltán,63.0,fuggetlen jelolt,69.230769,30.769231,38.461538,abaujlak,2014,local_elections,0,0,1
14,Gönczi Gáborné,61.0,fuggetlen jelolt,65.591398,29.032258,36.559140,abaujszolnok,2014,local_elections,0,0,1
18,Kiss Tibor István,646.0,fuggetlen jelolt,41.463415,28.626444,12.836970,abaujszanto,2014,local_elections,0,0,1
22,Sivák Tamás,69.0,fuggetlen jelolt,43.949045,41.401274,2.547771,abaujvar,2014,local_elections,0,0,1
23,Szabó Zsolt,890.0,fuggetlen jelolt,77.933450,22.066550,55.866900,abda,2014,local_elections,0,0,1


In [48]:
#saving data
elections_final2014.to_csv('elections_local_2014.csv', index = False)

## 2019

Election resulst are stored in 19 separate excel files by counties. Candidate names and parties and the results (by candidate id) are stored in a separated sheet.

In [49]:
import os
import glob
from textdistance import levenshtein
import unidecode
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd

In [50]:
os.chdir('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2019_onkormanyzati')

In [51]:
#reading results
results = pd.DataFrame()
for f in glob.glob("*.xls"):
    df = pd.read_excel(f, 'Polgármesterek választása')
    results = results.append(df,ignore_index=True)

In [52]:
#results.head(50)

In [53]:
#reading candidate names and parties
candidates = pd.DataFrame()
for f in glob.glob("*.xls"):
    xlsx = pd.ExcelFile(f)
    sheet1= xlsx.parse(1)
    df = sheet1
    candidates = candidates.append(df,ignore_index=True)

In [54]:
#candidates

In [55]:
candidates.loc[candidates.Település == 'Kömlő', 'Település'] = "komlo2"
candidates.loc[candidates.Település == 'Kömörő', 'Település'] = "komoro2"
candidates= candidates.rename(columns = {'Település': 'village_name', 'Jelölt sorszáma': 'candidate_id',
                                             'Jelölt neve': 'candidate_name', 'Jelölő szervezet': 'party'}, inplace = False)
candidates['village_name'] = candidates['village_name'].str.lower()
candidates['village_name'] = candidates['village_name'].apply(unidecode.unidecode)
#candidates

In [56]:
#budapest1 = candidates.loc[lambda x: x['village_name'].str.contains('budapest', regex = True)]
#budapest1

In [57]:
#saving data
#results.to_csv('elections_2019_raw.csv', index = False, encoding='utf-8')

#result data was manually cleaned and checked.
#saved to: elections_2019_raw_manually_cleaned

In [58]:
results = pd.read_csv('C:/Users/User/Documents/Thesis/Data/raw_data/election_results/2019_onkormanyzati/elections_2019_raw_manually_cleaned2.csv')
#results

In [59]:
total_votes = results.groupby(['village_name']).sum().reset_index()
#total_votes['id'] = total_votes.index
total_votes

,village_name,total_votes,1,2,3,4,5,6,7,8,9
0,Aba,2125,1369,523,233,0,0,0.0,0.0,0.0,0.0
1,Abaliget,361,22,9,34,115,3,48.0,130.0,0.0,0.0
2,Abasár,1252,617,635,0,0,0,0.0,0.0,0.0,0.0
3,Abaújalpár,34,34,0,0,0,0,0.0,0.0,0.0,0.0
4,Abaújkér,342,223,119,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3170,Őrimagyarósd,152,43,18,25,48,18,0.0,0.0,0.0,0.0
3171,Őriszentpéter,430,430,0,0,0,0,0.0,0.0,0.0,0.0
3172,Őrtilos,266,0,21,105,62,21,6.0,4.0,47.0,0.0
3173,Ősagárd,124,124,0,0,0,0,0.0,0.0,0.0,0.0


In [60]:
total_votes.loc[total_votes.village_name == 'Kömlő', 'village_name'] = "komlo2"
total_votes.loc[total_votes.village_name == 'Kömörő', 'village_name'] = "komoro2"
total_votes['village_name'] = total_votes['village_name'].str.lower()
total_votes['village_name'] = total_votes['village_name'].apply(unidecode.unidecode)
total_votes['village_name'] = total_votes['village_name'].str.strip()
total_votes

,village_name,total_votes,1,2,3,4,5,6,7,8,9
0,aba,2125,1369,523,233,0,0,0.0,0.0,0.0,0.0
1,abaliget,361,22,9,34,115,3,48.0,130.0,0.0,0.0
2,abasar,1252,617,635,0,0,0,0.0,0.0,0.0,0.0
3,abaujalpar,34,34,0,0,0,0,0.0,0.0,0.0,0.0
4,abaujker,342,223,119,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3170,orimagyarosd,152,43,18,25,48,18,0.0,0.0,0.0,0.0
3171,oriszentpeter,430,430,0,0,0,0,0.0,0.0,0.0,0.0
3172,ortilos,266,0,21,105,62,21,6.0,4.0,47.0,0.0
3173,osagard,124,124,0,0,0,0,0.0,0.0,0.0,0.0


In [61]:
total_votes.loc[total_votes.village_name == 'budapest 01', 'village_name'] = "budapest i. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 02', 'village_name'] = "budapest ii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 03', 'village_name'] = "budapest iii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 04', 'village_name'] = "budapest iv. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 05', 'village_name'] = "budapest v. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 06', 'village_name'] = "budapest vi. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 07', 'village_name'] = "budapest vii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 08', 'village_name'] = "budapest viii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 09', 'village_name'] = "budapest ix. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 10', 'village_name'] = "budapest x. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 11', 'village_name'] = "budapest xi. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 12', 'village_name'] = "budapest xii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 13', 'village_name'] = "budapest xiii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 14', 'village_name'] = "budapest xiv. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 15', 'village_name'] = "budapest xv. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 16', 'village_name'] = "budapest xvi. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 17', 'village_name'] = "budapest xvii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 18', 'village_name'] = "budapest xviii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 19', 'village_name'] = "budapest xix. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 20', 'village_name'] = "budapest xx. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 21', 'village_name'] = "budapest xxi. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 22', 'village_name'] = "budapest xxii. kerulet"
total_votes.loc[total_votes.village_name == 'budapest 23', 'village_name'] = "budapest xxiii. kerulet"

In [62]:
total_votes2 = total_votes[['village_name', 'total_votes']]

In [63]:
#total_long = pd.melt(total_votes, id_vars=['village_name'], 
#                     value_vars=['1_cand','2_cand','3_cand','4_cand','5_cand','6_cand','7_cand','8_cand','9_cand'])
total_long = pd.melt(total_votes, id_vars=['village_name'], 
                     value_vars=['1','2','3','4','5','6','7','8','9'])

In [64]:
total_long['variable']=total_long['variable'].astype(int)
#total_long['village_name'] = total_long['village_name'].str.encode('utf-8')
#candidates['village_name'] = candidates['village_name'].str.encode('utf-8')
total_long['village_name'] = total_long['village_name'].astype(str)
total_long['village_name'] = total_long['village_name'].str.strip()
#candidates['village_name'] = candidates['village_name'].astype(str)
#total_long['village_name'] = total_long['village_name'].str.encode('utf-8')
#candidates['village_name'] = candidates['village_name'].str.encode('utf-8')

#candidates['candidate_id']=candidates['candidate_id'].astype(object)

In [65]:
#total_long

In [66]:
#candidates

In [67]:
total_long['variable']=total_long['variable'].astype(int)

In [68]:
#print(total_long.dtypes)
#print(candidates.dtypes)

In [69]:
total_votes_merged = candidates.merge(total_long, how='left', left_on=['village_name', "candidate_id"], right_on=['village_name', "variable"])
total_votes_merged = total_votes_merged.merge(total_votes2, how='left', left_on=['village_name'], right_on=['village_name'])

In [70]:
#total_votes_merged

In [71]:
#maximum
forMax = total_votes_merged[[ "village_name", "value"]]
maximums = forMax.groupby([ 'village_name']).max().reset_index()
maximums['mandate'] = 1
#maximums

In [72]:
#calculating votes share
total_votes3_merged = total_votes_merged.merge(maximums, how='left', on=['village_name', "value"])
total_votes3_merged['votes_percent'] =total_votes3_merged['value']/total_votes3_merged['total_votes']*100
#total_votes3_merged

In [73]:
#calculating the percentage of votes each candidate got
margin = total_votes3_merged[['village_name','candidate_name','party', "value", "total_votes", "mandate", 'votes_percent']]
margin = margin.loc[margin['votes_percent'] != 100]
margin = margin.loc[margin['votes_percent'] != 0]
margin_2nd =pd.DataFrame(margin.groupby(['village_name'])['votes_percent'].nlargest(2).reset_index())
margin_2nd['second_share'] = margin_2nd['votes_percent'] 
margin_2nd['second_share_copy'] = margin_2nd['votes_percent'] 
margin_2nd['second_share'] = margin_2nd['second_share'].shift(-1)
margin_2nd['mask'] = margin_2nd.duplicated(margin_2nd.columns[[0,1]])
margin_2nd = margin_2nd.loc[margin_2nd['mask'] != True]
##margin_2nd = margin_2nd[margin_2nd.index % 2 == 0]
margin_2nd["margin"] = margin_2nd['votes_percent'] -margin_2nd['second_share']
margin_2nd['flag']=1
#margin_2nd = margin_2nd[['county_code_x','village_code','votes_percent', 'second_share','margin','flag']]
#margin_2nd

In [74]:
#merge margin of victory
elections_final = total_votes3_merged.merge(margin_2nd, how='left', on=['village_name', 'votes_percent'])

#only keep winners
elections_final = elections_final.loc[elections_final["mandate"] == 1]

#removing redundant columns
elections_final = elections_final.drop(columns=['level_1', 'second_share_copy', 'mask', 'flag', 'candidate_id', 'variable'])

#cleaning village names
elections_final.loc[elections_final['votes_percent'] == 100, 'margin'] = elections_final['votes_percent']
elections_final = elections_final.rename(columns = {3: 'votes_winner', 'votes_percent': 'votes_share_winner'}, inplace = False)
elections_final['year'] = 2019
elections_final['election_type'] = 'local_elections'
elections_final.head(50)

#elections_final.loc[elections_final['village_name']=='budapest i. kerulet']


,village_name,candidate_name,party,value,total_votes,mandate,votes_share_winner,second_share,margin,year,election_type
2,budapest i. kerulet,VÁRADINÉ NASZÁLYI MÁRTA,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,6025.0,12471.0,1.0,48.312084,47.341833,0.970251,2019,local_elections
3,budapest ii. kerulet,ÖRSI GERGELY FERENC,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,21557.0,40934.0,1.0,52.662823,47.337177,5.325646,2019,local_elections
5,budapest iii. kerulet,DR. KISS LÁSZLÓ,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,27206.0,53955.0,1.0,50.423501,49.576499,0.847002,2019,local_elections
10,budapest iv. kerulet,DÉRI TIBOR,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,20174.0,39946.0,1.0,50.503179,45.704201,4.798979,2019,local_elections
12,budapest v. kerulet,SZENTGYÖRGYVÖLGYI PÉTER,FIDESZ-MAGYAR POLGÁRI SZÖVETSÉG-KERESZTÉNYDEMO...,5450.0,10383.0,1.0,52.489647,42.473274,10.016373,2019,local_elections
16,budapest vi. kerulet,SOPRONI TAMÁS ISTVÁN,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,7314.0,12723.0,1.0,57.486442,42.513558,14.972884,2019,local_elections
20,budapest vii. kerulet,NIEDERMÜLLER PÉTER,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,8279.0,17283.0,1.0,47.902563,41.040329,6.862235,2019,local_elections
25,budapest viii. kerulet,PIKÓ ANDRÁS BRUNÓ,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,11241.0,22662.0,1.0,49.602859,48.415850,1.187009,2019,local_elections
28,budapest ix. kerulet,BARANYI KRISZTINA,MOMENTUM MOZGALOM-DEMOKRATIKUS KOALÍCIÓ-MAGYAR...,12188.0,21186.0,1.0,57.528557,39.535542,17.993014,2019,local_elections
33,budapest x. kerulet,D. KOVÁCS RÓBERT ANTAL,FIDESZ-MAGYAR POLGÁRI SZÖVETSÉG-KERESZTÉNYDEMO...,12569.0,24416.0,1.0,51.478539,44.732962,6.745577,2019,local_elections


In [75]:
elections_final['party'] = elections_final['party'].str.lower()
elections_final['party'] = elections_final['party'].apply(unidecode.unidecode)
elections_final['party'] = elections_final['party'].str.strip()
# create a choice list
choices = elections_final['party'].values.tolist()
#Remove(choices)

In [76]:
#final cleaning
elections_final2019=elections_final.copy()
elections_final2019= elections_final2019[['candidate_name', 'total_votes', 'party', 'votes_share_winner', 'second_share', 'margin', 'village_name', 'year', 'election_type']]
elections_final2019 = elections_final2019.rename(columns = {'total_votes': 'votes_winner', 'votes_share_winner': 'winner_share', 
                                                              'party':'party_name'}, inplace = False)
elections_final2019['village_name'] = elections_final2019['village_name'].str.replace("kerulet", "", case = False)
fidesz = pd.Series([
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-nemzeti forum egyesulet',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-kalocsa jovojeert egyesulet',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-nemzeti forum egyesulet-elheto kiskunmajsa egyesulet',
 'nemzeti forum egyesulet-fidesz-magyar polgari szovetseg-keresztenydemokrata neppart',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-jobboldali osszefogas bekesert polgari kor',
 'fidesz-magyar polgari szovetseg',
 'mindenki magyarorszaga mozgalom',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-"putnok varosert" egyesulet',
 'keresztenydemokrata neppart-fidesz-magyar polgari szovetseg',
 'martonvasari polgari osszefogas egyesulet-fidesz-magyar polgari szovetseg-keresztenydemokrata neppart',
 'polgari osszefogas perkataert egyesulet-fidesz-magyar polgari szovetseg-keresztenydemokrata neppart',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-szeretem polgardit egyesulet',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-kisujszallasi nagykun polgari kor',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-a magyar vallalkozok es munkaadok partja',
 'keresztenydemokrata neppart',
 'fidesz-magyar polgari szovetseg-keresztenydemokrata neppart-balaicz zoltan jeloltje egyesulet'])
independent = pd.Series(['fuggetlen jelolt'])
elections_final2019['fidesz_list'] = pd.Categorical(elections_final2019['party_name'].isin(fidesz).astype(np.int8))
elections_final2019['governing_pary'] = pd.Categorical(elections_final2019['party_name'].isin(fidesz).astype(np.int8))
elections_final2019['independent'] = pd.Categorical(elections_final2019['party_name'].isin(independent).astype(np.int8))
elections_final2019.head(50)

,candidate_name,votes_winner,party_name,winner_share,second_share,margin,village_name,year,election_type,fidesz_list,governing_pary,independent
2,VÁRADINÉ NASZÁLYI MÁRTA,12471.0,momentum mozgalom-demokratikus koalicio-magyar...,48.312084,47.341833,0.970251,budapest i.,2019,local_elections,0,0,0
3,ÖRSI GERGELY FERENC,40934.0,momentum mozgalom-demokratikus koalicio-magyar...,52.662823,47.337177,5.325646,budapest ii.,2019,local_elections,0,0,0
5,DR. KISS LÁSZLÓ,53955.0,momentum mozgalom-demokratikus koalicio-magyar...,50.423501,49.576499,0.847002,budapest iii.,2019,local_elections,0,0,0
10,DÉRI TIBOR,39946.0,momentum mozgalom-demokratikus koalicio-magyar...,50.503179,45.704201,4.798979,budapest iv.,2019,local_elections,0,0,0
12,SZENTGYÖRGYVÖLGYI PÉTER,10383.0,fidesz-magyar polgari szovetseg-keresztenydemo...,52.489647,42.473274,10.016373,budapest v.,2019,local_elections,1,1,0
16,SOPRONI TAMÁS ISTVÁN,12723.0,momentum mozgalom-demokratikus koalicio-magyar...,57.486442,42.513558,14.972884,budapest vi.,2019,local_elections,0,0,0
20,NIEDERMÜLLER PÉTER,17283.0,momentum mozgalom-demokratikus koalicio-magyar...,47.902563,41.040329,6.862235,budapest vii.,2019,local_elections,0,0,0
25,PIKÓ ANDRÁS BRUNÓ,22662.0,momentum mozgalom-demokratikus koalicio-magyar...,49.602859,48.415850,1.187009,budapest viii.,2019,local_elections,0,0,0
28,BARANYI KRISZTINA,21186.0,momentum mozgalom-demokratikus koalicio-magyar...,57.528557,39.535542,17.993014,budapest ix.,2019,local_elections,0,0,0
33,D. KOVÁCS RÓBERT ANTAL,24416.0,fidesz-magyar polgari szovetseg-keresztenydemo...,51.478539,44.732962,6.745577,budapest x.,2019,local_elections,1,1,0


In [77]:
#saving data
elections_final2019.to_csv('elections_local_2019.csv', index = False)

#### merging all local elections data

In [78]:
elections_final = pd.concat([elections_final2002, elections_final2006, elections_final2010, elections_final2014,
                                 elections_final2019])

In [79]:
elections_final

,candidate_name,votes_winner,party_name,winner_share,second_share,margin,village_name,year,election_type,fidesz_list,mszp_list,governing_pary,independent
2,NAGY GÁBOR TAMÁS,7247.0,fidesz-magyar polgari part,46.830372,34.862682,11.967690,budapest i.,2002,local_elections,1,0,0,0
6,HORVÁTH CSABA,22766.0,magyar szocialista part,47.059553,46.912789,0.146764,budapest ii.,2002,local_elections,0,1,1,0
9,TARLÓS ISTVÁN,30509.0,fuggetlen,52.779171,43.755730,9.023441,budapest iii.,2002,local_elections,0,0,0,1
11,DERCE TAMÁS,23151.0,ujpestert egyesulet,58.553796,41.446204,17.107593,budapest iv.,2002,local_elections,0,0,0,0
13,STEINER PÁL,7580.0,magyar szocialista part,48.366514,47.626340,0.740174,budapest v.,2002,local_elections,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7464,BÖRÖNDI FERENC,66.0,fuggetlen jelolt,74.242424,25.757576,48.484848,zalaujlak,2019,local_elections,0,NaN,0,1
7467,LUCZ JÓZSEF,423.0,fuggetlen jelolt,47.990544,31.205674,16.784870,zalavar,2019,local_elections,0,NaN,0,1
7469,MARTON ANDRÁS,168.0,fuggetlen jelolt,100.000000,NaN,100.000000,zalaveg,2019,local_elections,0,NaN,0,1
7470,KERESZTURI JÓZSEF,29.0,fuggetlen jelolt,100.000000,NaN,100.000000,zebecke,2019,local_elections,0,NaN,0,1


In [80]:
elections_final.to_csv('local_elections_final.csv', index = False)